# Kubernetes Services

# Exposing our application as a Service

## Service abstracts pods details
As mentioned earlier exposing our application as a Service allows to access the application at a single virtual ip address and port - without having to know the ip addresses of the individual pods which may change ip address as pods are scaled up/down or as pods or nodes fail or come on line.

It can also allow load-balancing across pods.

## Accessing our application - via a Service NodePort

We can expose our service using different types.

The first type we will consider is called NodePort and has the effect of exposing our Service on the same port on each Kubernetes node - and so with the ip address of that node.

Let's try that with our service:

In [99]:
kubectl delete svc/k8s-demo-service
kubectl expose deploy k8s-demo --type=NodePort --name=k8s-demo-service

service "k8s-demo-service" deleted
service "k8s-demo-service" exposed


In [100]:
kubectl get svc/k8s-demo-service -o wide

EXPOSED_PORT=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ FS=":"; $0=$5; FS="/"; $0=$2; print $1;}')
CLUSTER_IP=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ print $3;}')

echo EXPOSED_PORT=$EXPOSED_PORT
echo CLUSTER_IP=$CLUSTER_IP
echo MASTER_IP=$MASTER_IP


NAME               TYPE       CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE       SELECTOR
k8s-demo-service   NodePort   10.101.126.101   <none>        8080:31092/TCP   4s        app=k8s-demo
EXPOSED_PORT=31092
CLUSTER_IP=10.101.126.101
MASTER_IP=10.192.0.2


In [101]:
curl $MASTER_IP:$EXPOSED_PORT

curl: (7) Failed to connect to 10.192.0.2 port 31092: Connection refused


: 7

In [102]:
#kubectl get svc/k8s-demo-service -o wide
kubectl describe nodes | grep InternalIP:

#echo $MASTER_IP
#curl $MASTER_IP:$EXPOSED_PORT
#curl 10.105.128.87:$EXPOSED_PORT

curl 10.192.0.2:$EXPOSED_PORT
curl 10.192.0.3:$EXPOSED_PORT
curl 10.192.0.4:$EXPOSED_PORT

  InternalIP:  10.192.0.2
  InternalIP:  10.192.0.3
  InternalIP:  10.192.0.4
curl: (7) Failed to connect to 10.192.0.2 port 31092: Connection refused
curl: (7) Failed to connect to 10.192.0.3 port 31092: Connection refused
curl: (7) Failed to connect to 10.192.0.4 port 31092: Connection refused


: 7

In [103]:
kubectl describe svc/k8s-demo-service

Name:                     k8s-demo-service
Namespace:                default
Labels:                   app=k8s-demo
Annotations:              <none>
Selector:                 app=k8s-demo
Type:                     NodePort
IP:                       10.101.126.101
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  31092/TCP
Endpoints:                10.192.2.1:8080,10.192.3.1:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [104]:
curl 10.101.126.101:32640

## Accessing our application - via a Service LoadBalancer


In [105]:
kubectl delete svc/k8s-demo-service
kubectl expose deploy k8s-demo --type=LoadBalancer --name=k8s-demo-service

service "k8s-demo-service" deleted
service "k8s-demo-service" exposed


In [106]:
kubectl get svc/k8s-demo-service -o wide

NAME               TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE       SELECTOR
k8s-demo-service   LoadBalancer   10.106.67.139   <pending>     8080:30298/TCP   12s       app=k8s-demo


In [109]:
kubectl describe svc/k8s-demo-service

EXPOSED_PORT=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ FS=":"; $0=$5; FS="/"; $0=$2; print $1;}')
CLUSTER_IP=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ print $3;}')

echo
echo EXPOSED_PORT=$EXPOSED_PORT
echo CLUSTER_IP=$CLUSTER_IP
echo MASTER_IP=$MASTER_IP

Name:                     k8s-demo-service
Namespace:                default
Labels:                   app=k8s-demo
Annotations:              <none>
Selector:                 app=k8s-demo
Type:                     LoadBalancer
IP:                       10.106.67.139
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  30298/TCP
Endpoints:                10.192.2.1:8080,10.192.3.1:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>

EXPOSED_PORT=30298
CLUSTER_IP=10.106.67.139
MASTER_IP=10.192.0.2


In [ ]:
curl ${CLUSTER_IP}:$EXPOSED_PORT

In [111]:
curl ${MASTER_IP}:$EXPOSED_PORT

curl: (7) Failed to connect to 10.192.0.2 port 30298: Connection refused


: 7

In [112]:
kubectl describe no/kube-master | grep "InternalIP:"

MASTER_IP=$(kubectl describe no/kube-master | grep "InternalIP:" | sed -e 's/.*: *//')
echo MASTER_IP=$MASTER_IP
WORKER1_IP=$(kubectl describe no/kube-node-1 | grep "InternalIP:" | sed -e 's/.*: *//')
echo WORKER1_IP=$WORKER1_IP
WORKER2_IP=$(kubectl describe no/kube-node-2 | grep "InternalIP:" | sed -e 's/.*: *//')
echo WORKER2_IP=$WORKER2_IP

#curl $MASTER_IP:8080
curl $WORKER2_IP:8080
curl $WORKER2_IP:$EXPOSED_PORT
curl $WORKER1_IP:8080
curl $WORKER1_IP:$EXPOSED_PORT
curl $MASTER_IP:$EXPOSED_PORT


  InternalIP:  10.192.0.2
MASTER_IP=10.192.0.2
WORKER1_IP=10.192.0.3
WORKER2_IP=10.192.0.4
curl: (7) Failed to connect to 10.192.0.4 port 8080: Connection refused
curl: (7) Failed to connect to 10.192.0.4 port 30298: Connection refused
curl: (7) Failed to connect to 10.192.0.3 port 8080: Connection refused
curl: (7) Failed to connect to 10.192.0.3 port 30298: Connection refused
curl: (7) Failed to connect to 10.192.0.2 port 30298: Connection refused


: 7